## Create Z500

In [1]:
import sys
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
#import collections
import pandas as pd
import matplotlib.ticker as mticker
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

# import netCDF4
# from netCDF4 import *

import cartopy.crs as ccrs
import cartopy as cart
import cartopy.mpl.ticker as cticker
import cartopy.feature as cfeature
from scipy import interpolate
from scipy.interpolate import griddata
import time
import glob
import dask

from scipy.fftpack import fft
from scipy.fftpack import ifft
import copy
import eofs.standard as Eof_st
from matplotlib.colors import ListedColormap

from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter

import cmocean

In [2]:
if 'client' in locals():
    client.shutdown()
    print('...shutdown client...')
else:
    print('client does not exist yet')

client does not exist yet


In [ ]:
from distributed import Client
from ncar_jobqueue import NCARCluster

cluster = NCARCluster(project='NAML0001',walltime='12:00:00')
cluster.scale(40)
client = Client(cluster)
client

## Useful Functions

In [4]:
#this functions pick data ranges out of an xarray data structure.
def is_dayofyear(dayofyear,dd):
    return (dayofyear==dd)

def is_dayofyear_range(dayofyear,dd1,dd2):
    
    if (dd2 > dd1) & (dd2<366) & (dd1>=1):
        return ((dayofyear>=dd1)&(dayofyear<=dd2))
    elif (dd2 > 366) & (dd1>0):
        dd2 = dd2 - 366
        print('you are wrapping around years... this is a warning if you arent intending to')
        return ((dayofyear>=dd1)|(dayofyear<=dd2))
    elif dd1 < 1:
        print('you are wrapping around years... this is a warning if you arent intending to')
        dd1 = dd1+366
        return ((dayofyear>=dd1)|(dayofyear<=dd2))
    else:
        print('this is an edge case you did not foresee')
        return np.nan
    
def cal_ano_dcli(var):
    '''return [var_ano, var_dcli], Taxis= axis of time'''
    var_dcli=var.groupby('time.dayofyear').mean(dim='time')
    var_ano= var - var.mean(dim = 'time')
    
    return var_ano, var_dcli




#make MJO base line months
def is_NDJFM(dayofyear):
    return ((dayofyear==11)|(dayofyear==12)|(dayofyear==1)|(dayofyear==2)|(dayofyear==3))



def is_doyrange(doy,dd,tod,hh):
    daywind = 15
    if (dd - daywind) < 1:
        # print('in 1')
        return ((doy >= (366+(dd-daywind))) | (doy <= dd+daywind)) & (tod==hh)
        
    elif (dd + daywind) > 366:
        # print('in 2')
        return ((((doy <= 366) & (doy>=(dd-daywind))) | (doy <= (-1*((366-(dd-daywind))-(2*daywind)))  )))&(tod==hh)
    
    else:
        # print('in 3')
        return ((doy >= dd-daywind) & (doy <= dd+daywind))&(tod==hh)
    

# def is_NDJFM(dayofyear):
#     return ((dayofyear==12)|(dayofyear==1)|(dayofyear==2))

# def is_NDJFM(dayofyear):
#     return ((dayofyear==1)|(dayofyear==2)|(dayofyear==3)|(dayofyear==4)|(dayofyear==5)|(dayofyear==6)|(dayofyear==7)|(dayofyear==8)|(dayofyear==9)|(dayofyear==10)|(dayofyear==11)|(dayofyear==12))

#  NDJFM_data = GPH_E20c.sel(time=is_NDJFM(GPH_E20c['time.month']))

## First Load the CESM LENS Data
need Z500


In [12]:
%%time
FNS_all = sorted(glob.glob('/glade/scratch/wchapman/ERA5_uvolr/1deg/*128_129_z*'))
DSz500 =  xr.open_mfdataset(FNS_all,parallel=True)

DSz500 = DSz500.chunk(chunks={'time':100,'lat':192,'lon':288})

DSz500=DSz500.convert_calendar("standard", use_cftime=True)
print('....loading....')
DSz500=DSz500.load()

In [22]:
%%time
def cal_ano_dcli(var):
    '''return [var_ano, var_dcli], Taxis= axis of time'''
    var_dcli=var.groupby('time.dayofyear').mean(dim='time')
    var_ano= var.groupby('time.dayofyear') - var_dcli    
    return xvar_ano, var_dcli

DSz500_anom,DSz500_clim = cal_ano_dcli(DSz500)
print('....adding states...')
DSz500['Z500_anom']=DSz500_anom['Z500']
DSz500['Z500_clim'] = DSz500['Z500']-DSz500['Z500_anom']
DSz500.to_netcdf('/glade/scratch/wchapman/data_for_KJM/'+'Z500_CESM2_BHIST_100_400_KJMmethod.nc')

## Make Climatology
- This takes about 1000 years.. it's better to just load the files if they have already been made

In [22]:
%%time

def is_doyrange(doy,dd,tod,hh):
    daywind = 15
    if (dd - daywind) < 1:
        # print('in 1')
        return ((doy >= (366+(dd-daywind))) | (doy <= dd+daywind)) & (tod==hh)
        
    elif (dd + daywind) > 366:
        # print('in 2')
        return ((((doy <= 366) & (doy>=(dd-daywind))) | (doy <= (-1*((366-(dd-daywind))-(2*daywind)))  )))&(tod==hh)
    
    else:
        # print('in 3')
        return ((doy >= dd-daywind) & (doy <= dd+daywind))&(tod==hh)


Z500_climo = xr.zeros_like(DSz500['Z'])
print('... creating climo from centered 30 day average ...')
for ee,dayhr in (enumerate(DSz500.time)):
    if ee%1 ==0:
        print('doing ',ee,' of 365')
    dooDOY = dayhr['time.dayofyear']
    hh=dayhr['time.hour']
    Dtemp = DSz500.sel(time=is_doyrange(DSz500['time.dayofyear'],dooDOY,DSz500['time.hour'],hh))[['Z']].mean(['time'])
    Z500_climo[ee,:,:] = Dtemp['Z'].values
    
    if ee == (365)+8:
        endee=ee
        enddate = dayhr
        break

print("this is much faster but only works with noleap calendar:")  
svname = '/glade/scratch/wchapman/data_for_KJM//'+'Z500_ERA5_1975_2022_WEC_repeats.nc'
DSz500.to_netcdf(svname)
print('otherwise you have to do this: (uncomment below)')

print('...now repeating climo...') 
for ee,dayhr in (enumerate(DSz500.time)):
    dm = dayhr['time.month'].values
    dd = dayhr['time.day'].values
    dy = DSz500['time.year'][0].values

    getday = f'{dy:04}'+'-'+f'{dm:02}'+'-'+f'{dd:02}'
    
    #leap year shenanigans:
    if f'{dm:02}'+'-'+f'{dd:02}' == '02-29': 
        getday = f'{dy:04}'+'-02-28'
        
    if ee%500 ==0:
        print('doing ',ee,' of ', len(DSz500['time']))
    try:
        DtempZ500 = Z500_climo.sel(time=getday).squeeze()
    except: 
        DtempZ500 = Z500_climo.sel(time=getday,method='nearest').squeeze()
    Z500_climo[ee,:,:] = DtempZ500.values
    
DSz500['Z500_climo'] = Z500_climo

svname = '/glade/scratch/wchapman/data_for_KJM/'+'Z500_ERA5_1975_2022_WEC.nc'
DSz500.to_netcdf(svname)

...now repeating climo...
doing  0  of  17897
doing  500  of  17897
doing  1000  of  17897
doing  1500  of  17897
doing  2000  of  17897
doing  2500  of  17897
doing  3000  of  17897
doing  3500  of  17897
doing  4000  of  17897
doing  4500  of  17897
doing  5000  of  17897
doing  5500  of  17897
doing  6000  of  17897
doing  6500  of  17897
doing  7000  of  17897
doing  7500  of  17897
doing  8000  of  17897
doing  8500  of  17897
doing  9000  of  17897
doing  9500  of  17897
doing  10000  of  17897
doing  10500  of  17897
doing  11000  of  17897
doing  11500  of  17897
doing  12000  of  17897
doing  12500  of  17897
doing  13000  of  17897
doing  13500  of  17897
doing  14000  of  17897
doing  14500  of  17897
doing  15000  of  17897
doing  15500  of  17897
doing  16000  of  17897
doing  16500  of  17897
doing  17000  of  17897
doing  17500  of  17897
CPU times: user 9min 21s, sys: 3.83 s, total: 9min 25s
Wall time: 9min 40s


## Hacky Climo Way

In [16]:
Z500S_climo = xr.open_dataset('/glade/scratch/wchapman/data_for_KJM//'+'Z500_CESM2_100_400_climo.nc')
DSjustclim = Z500S_climo['Z500'][:380,:,:]
Z500_climo = xr.zeros_like(DSz500['Z500'])

In [17]:
print("this is much faster but only works with noleap calendar:")  
Z500np = np.array(DSjustclim[:365,:,:])
print('...expand...')
arrays_to_concat = [Z500np] * 300
print('...done expand...')
concatenated_arr = np.concatenate(arrays_to_concat, axis=0)
Z500_climo[:,:,:]=concatenated_arr
dr = pd.date_range(start='1801-01-01', end='2100-12-31', freq='D')
dates = dr[(dr.day != 29) | (dr.month != 2)]
DSz500['Z500_climo'] = Z500_climo
DSz500['time']=dates
print('...saving...')
svname = '/glade/scratch/wchapman/data_for_KJM//'+'Z500_CESM2_100_400_repeats.nc'
DSz500.to_netcdf(svname)

this is much faster but only works with noleap calendar:
...expand...
...done expand...
...saving...


## For Tony to do.. Create your anomaly: 
- subtract Z500_climo from Z500 and save a new variable Z500a
- Take and average over the north Pacific (lat[20,70],lon[150-240])
- - This will make the "index" of interest